## Experiment Tracking

- Using T8 as Encoder
    - Not pre-trained -> accuracy 4%
    - pre-trained -> accuracy 64%
- Conv Encoder v1 -> Eliminated
    - catboost classifier 27% accuracy, 2048 embedding space, 30 epochs
    - catboost classifier 27% accuracy, 2048 embedding space, 60 epochs
    - encoder classifier 14% accuracy, 2048 embedding space, 60 epochs
- Conv Encoder v2
    - 768 embedding space, 30 epochs, catboost classifier 53% acc, 1000 iterations
    - 768 embedding space, 60 epochs, catboost classifier 55% acc, 3000 iterations
    - 1500 embedding space, 30 epochs, catboost classifier 51% acc, 1000 iterations
    - 512 embedding space, 30 epochs, catboost classifier 53.7% acc, 1000 iterations, lr=0.01
    - 512 embedding space, 60 epochs, catboost classifier 55.6%, 3000 iterations. lr=0.01
- Conv Encoder v3
    - 1024 embedding space, 30 epochs, catboost classifier 48% acc, 3000 iterations
    - 768 embedding space, 30 epochs, catboost classifier 51% acc, 1000 iterations
    - 512 embedding space, 30 epochs, catboost classifier ? acc,
- VitEncoder
    - 768 embedding space, 30 epochs, lr=0.01, catboost 22% acc, 1000 iterations
    - 768 embedding space, 70 epochs, lr=0.001 (with adamw),
- Conv Encoder v4
    - 768 embedding space, 40 epochs, lr=0.01, catboost 54% acc, 2000 iterations
    - 768 embedding space, 70 epochs, lr=0.01, catboost 58% acc, 3000 iterations ----------------- 39% on KAggle
    - 1024 embedding space, 300 epochs, lr=0.01, catboost 48% acc, 3000 iterations --------------- 30% on KAggle
- resnet 18 (pretrained=False) with new fc -> horror
    - 100 embedding size, 2%, final fc
    - 1000 embedding size, final fc
- resnet embedding -> horror
- SimpleCNN v2
    - 1200 embedding size, 40 epochs, lr = 0.001, bs=128, catboost 42% acc, 3000 iterations
- Simple CNN v1
    - 1024 embedding size, 100 epochs, lr=0.01, catboost 4% , 3000 iterations, -> horror
- Conv Encoder v5
    - 4096 embedding size, 26 epochs max, 26% catboost accuracy 

- Conv v4 + Deep Encoder (from Autoencoder) -> on the catboost trained on train, inserted validation for params, but did not also train on valid
    - Conv v4 (768) + deep enc v1 (2048) -> 61% catboost accuracy, -------------------- 56% kaggle
    - Conv v2 (768, 60e) + Conv v4 (768, 70e) + deep enc v1 (2048) -> 55%% catboost accuracy, -> v2 seems to drop the performance
    - Conv v3 (768, 70e) + Conv v4 (768, 70e) + Deep Encoder V1(2048) -> 64.7% accuracy ---------------------------- 61% on kaggle
    - Conv v3 (768, 70e) + Conv v4 (768, 70e) + Deep Encoder V1(2048) + SimpleCNN v2 (1200, 40 epochs) -> 64.4% accuracy (2000 iterations, could probably get out a bit more through more iterations)  -------------
    - Conv v3 (768, 70e) + Conv v4 (768, 70e) + Deep Encoder V1(2048) + SimpleCNN v2 (1200, 40 epochs) + Deep Enc CLF (train + valid) -> 97% accuracy------------- 50% Kaggle WTFFFFF
    - Conv v3 (768, 70e) + Conv v4 (768, 70e) + Deep Encoder V1(2048) + SimpleCNN v2 (1200, 40 epochs) + Deep Enc CLF (train) -> 50 % accuracy------------- no point in uploading to Kaggle
    - Conv v3 (768, 70e) + Conv v4 (768, 70e) + Deep Encoder V1(2048) + SimpleCNN v2 (1200, 40 epochs) + Novel Encoder (train) -> 63% accuracy -> 59% kaggle
    - Conv v3 (768, 70e) + Conv v4 (768, 70e) + Deep Encoder V1(2048) + SimpleCNN v2 (1200, 40 epochs) + mobilenetv3 small (train) -> 62.90% (1000 iter)
    - Conv v3 (768, 70e) + Conv v4 (768, 70e) + Deep Encoder V1(2048) + SimpleCNN v2 (1200, 40 epochs) + efficientnet b3 (train) -> 63% (2000 iter) (b3 pare ca are embeddings total useless, catboost nu invata nimic de pe urma lor)
- Ablation Studies
    - Conv v3 + conv v4 + Simple CNN v2 -> 62.8% catboost (1000 iter + eval)
    - deep encoder v1, simple cnn v2 -> 38.9 catboost (1000 iter + eval) -> pare ca deep encoder v1 strica tot, nu poarta aparent nicio valoare
    - mobilenet v3 large -> catboost (1000 iter + eval)
    - simplecnn v2 + conv v3 + conv v4-> catboost ? (1000 iter + eval)
    - simplecnn v2 + conv v3 + convv4 + mobilenet v3 large -> catboost 61.25% (2000 iter + eval) -> 

In [5]:
import timm
import torch
import torch.optim as optim
import torchvision.utils as vutils
from torchvision import models
import torchvision.transforms as T
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.nn import functional as F
import numpy as np
import pandas as pd
import os
from PIL import Image

import math
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, classification_report

from catboost import CatBoostClassifier

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [26]:
from custom_classifiers import ConvEncoderV3, ConvEncoderV4, ConvEncoderV5, SimpleCNNV2
from utils import CustomDataset

In [7]:
torch.cuda.empty_cache()
import gc
gc.collect()

0

In [8]:
def imshow(img, mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)):
    """Imshow for Tensor."""
    img = img.numpy().transpose((1, 2, 0))
    # img = std * img + mean  # unnormalize
    # img = np.clip(img, 0, 1)  # clip any values outside the range [0, 1]
    plt.imshow(img)
    plt.show()

### EDA

In [9]:
train_df = pd.read_csv('dataset/train.csv')
valid_df = pd.read_csv('dataset/val.csv')
test_df = pd.read_csv('dataset/test.csv')

print(f"Train: {train_df.shape}; Valid: {valid_df.shape}; Test: {test_df.shape}")

Train: (13000, 2); Valid: (2000, 2); Test: (5000, 1)


In [10]:
train_df.columns, train_df.head()

(Index(['Image', 'Class'], dtype='object'),
                                       Image  Class
 0  0be195e0-eb16-4f29-ac7c-196dec9da47d.png     79
 1  28045419-b3b2-415b-9085-b4d241944235.png     94
 2  b7078f35-d239-4dd6-babb-1af7be1b9364.png     79
 3  0f54f663-2953-432b-bdd4-9b9f7a78bfb9.png     23
 4  ba11dda2-37d7-4d28-8bbb-128d452a171c.png     88)

In [11]:
train_path = 'dataset/train_images/'
val_path = 'dataset/val_images/'
test_path = 'dataset/test_images/'

In [12]:
train_image_path = os.path.join(train_path,train_df['Image'][0])
print(train_image_path)

dataset/train_images/0be195e0-eb16-4f29-ac7c-196dec9da47d.png


In [13]:
image = Image.open(train_image_path)

In [14]:
np.array(image).shape

(64, 64, 3)

### Data Setup

In [15]:
train_image_names = train_df['Image'].tolist()
train_image_labels = train_df['Class'].tolist()
# print(train_image_names[:5], train_image_labels[:5])

train_image_paths = [os.path.join(train_path, image_name)for image_name in train_image_names]
print(train_image_paths[:5], train_image_labels[:5])

['dataset/train_images/0be195e0-eb16-4f29-ac7c-196dec9da47d.png', 'dataset/train_images/28045419-b3b2-415b-9085-b4d241944235.png', 'dataset/train_images/b7078f35-d239-4dd6-babb-1af7be1b9364.png', 'dataset/train_images/0f54f663-2953-432b-bdd4-9b9f7a78bfb9.png', 'dataset/train_images/ba11dda2-37d7-4d28-8bbb-128d452a171c.png'] [79, 94, 79, 23, 88]


In [16]:
val_image_names = valid_df['Image'].tolist()
val_image_labels = valid_df['Class'].tolist()

val_image_paths = [os.path.join(val_path, image_name)for image_name in val_image_names]
print(val_image_paths[:5], val_image_labels[:5])

['dataset/val_images/e91a8fbc-d3ba-4b39-8c2f-04c14de78e5e.png', 'dataset/val_images/7c40819b-c3ce-4a91-9e98-c3df11b63623.png', 'dataset/val_images/d54269d7-fe86-4112-9c0f-99cc6ab8d9c0.png', 'dataset/val_images/cbf9ac9e-0859-4b54-ae65-347587b45deb.png', 'dataset/val_images/6aafce3f-9002-44e0-9a99-ffe9b49c9bac.png'] [32, 85, 41, 97, 62]


In [17]:
test_image_names = test_df['Image'].tolist()
test_image_paths = [os.path.join(test_path, image_name)for image_name in test_image_names]
print(test_image_paths[:5])

['dataset/test_images/046f61c4-b825-459a-8b2d-07503f5b94a5.png', 'dataset/test_images/67db001f-e287-4950-ac49-6683b493d1a4.png', 'dataset/test_images/9f1d36a1-f046-4c5d-9e8a-0a3758ff605c.png', 'dataset/test_images/5ffef91a-aaf9-4d0d-a219-83a9f5282361.png', 'dataset/test_images/c00af570-0000-4f8f-a3f2-c37a981bfdb1.png']


### Load Encoder Architectures

In [18]:
# Example usage
encoder = ConvEncoderV3(embedding_dim=768)
encoder.load_state_dict(torch.load('conv_encoder_v3_768_30e_train.pth'))
encoder = encoder.to(device)

#### Conv Encoder v4

In [20]:
# Example usage
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder = ConvEncoderV4(embedding_dim=768)
encoder.load_state_dict(torch.load('conv_enc_v4_768_emb_70e_train.pth'))
encoder = encoder.to(device)

#### Simple CNN v2

In [21]:
encoder = SimpleCNNV2(embedding_dim=1200)
encoder.load_state_dict(torch.load('simple_cnn_v2_40e_emb_1200_train.pth'))
encoder = encoder.to(device)

#### Conv Encoder v5

In [22]:
# Example usage
encoder = ConvEncoderV5(embedding_dim = 4096)
encoder.load_state_dict(torch.load('conv_encoder_v5_checkpoint_epoch_21.pth')['encoder_state_dict'])
# input_image = torch.randn(1, 3, 64, 64) # Example input tensor (batch_size, channels, height, width)
# output = encoder(input_image)
# print(output.shape)  # Output shape after passing through the encoder
encoder = encoder.to(device)

### Get Embeddings

In [23]:
basic_transform = T.Compose([
    # T.Resize((64, 64)), # -> all are already 64 * 64
    T.ToTensor(),
    # T.Normalize([0.42835271, 0.40658227, 0.34071648], [0.2144312,  0.21884131, 0.20464434])
    T.Normalize((0.5,), (0.5,)),
])

In [24]:
# Function to process an image and get embedding
def get_embedding(image_path, encoder, flatten=True):
    image = Image.open(image_path)
    image = basic_transform(image).unsqueeze(0).to(device)  # Add batch dimension

    with torch.no_grad():
        embedding = encoder(image)
    
    if flatten:
        # Flatten the embedding
        embedding = torch.flatten(embedding, start_dim=0)
    
    return embedding

In [25]:
embedding = get_embedding(os.path.join(train_path, train_image_names[0]), encoder, flatten=True)
embedding.shape

torch.Size([4096])

### Train Encoder

In [15]:
train_transform = T.Compose([
    # T.RandomHorizontalFlip(p=0.5),
    # T.RandomVerticalFlip(p=0.5),
    # T.Resize((64, 64)), -> all are already 64 * 64
    T.ToTensor(),
    T.Normalize((0.5), (0.5))
])

In [15]:
train_dataset = CustomDataset(train_image_paths, train_image_labels, transform = train_transform)
train_data_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, pin_memory=True)

### Get Mean and Std for DataLoader, and create Transform with Augmentation

In [429]:
# def compute_mean_std(directory):
#     means, stds = [], []
#     for img_filename in os.listdir(directory):
#         img_path = os.path.join(directory, img_filename)
#         with Image.open(img_path) as img:
#             img_arr = np.array(img) / 255.0  # Normalize to [0, 1]
#             means.append(np.mean(img_arr, axis=(0, 1)))  # Mean across height and width
#             stds.append(np.std(img_arr, axis=(0, 1)))    # Std across height and width

#     mean = np.mean(means, axis=0)  # Mean across all images
#     std = np.mean(stds, axis=0)    # Std across all images

#     return mean, std

# mean, std = compute_mean_std(train_path)
# print(f"Mean: {mean}")
# print(f"Std: {std}")


Mean: [0.42835271 0.40658227 0.34071648]
Std: [0.2144312  0.21884131 0.20464434]


In [20]:
# Loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(encoder.parameters(), lr=0.001)

In [ ]:
%%time
losses = []
num_epochs = 50 
for epoch in range(num_epochs):
    encoder.train()
    running_loss = 0.0
    for images, labels in train_data_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        
        embeddings = encoder(images)
        loss = criterion(embeddings, labels)
        
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_data_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

    if epoch % 5 == 0 and epoch != 0:
        # Save checkpoint
        checkpoint = {
            'epoch': epoch + 1,
            'encoder_state_dict': encoder.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': losses
        }
        torch.save(checkpoint, f'vit_enc_v2_checkpoint_epoch_{epoch+1}.pth')

Epoch [1/50], Loss: 6.9631
Epoch [2/50], Loss: 6.9633


In [22]:
# TODO: Save encoder
torch.save(encoder.state_dict(), 'vit_encoder_v2_50e_emb_1024.pth')

In [24]:
encoder.load_state_dict(torch.load(f'conv_encoder_v5_checkpoint_epoch_21.pth')['encoder_state_dict'])
encoder = encoder.to(device)

In [27]:
# # To load for inference
# encoder = ImageEncoder(embedding_dim=1024)
# encoder.load_state_dict(torch.load('conv_enc_v4_300e_bs_128_emb_1024_train.pth'))
# encoder = encoder.to(device)
# encoder.eval()

### Training Data for CatBoost
Here we will load the embeddings generated by vision models, in an attempt at classifying images based on the embeddings generated.

In [17]:
train_embeddings = [get_embedding(img_path, encoder).cpu() for img_path in train_image_paths]
val_embeddings = [get_embedding(img_path, encoder).cpu() for img_path in val_image_paths]
test_embeddings = [get_embedding(img_path, encoder).cpu() for img_path in test_image_paths]

In [63]:
# import pickle
# # Load data (deserialize)
# with open('train_embeddings_mobilenet_v3_large.pkl', 'rb') as handle:
#     train_embeddings_ce = pickle.load(handle)

# # Load data (deserialize)
# with open('val_embeddings_mobilenet_v3_large.pkl', 'rb') as handle:
#     val_embeddings_ce = pickle.load(handle)

# # Load data (deserialize)
# with open('test_embeddings_mobilenet_v3_large.pkl', 'rb') as handle:
#     test_embeddings_ce = pickle.load(handle)

In [55]:
len(train_embeddings_ce[0])

2048

In [76]:
import pickle
# Load data (deserialize)
with open('train_embeddings_simple_cnn_v2.pkl', 'rb') as handle:
    train_embeddings_v2 = pickle.load(handle)

# Load data (deserialize)
with open('val_embeddings_simple_cnn_v2.pkl', 'rb') as handle:
    val_embeddings_v2 = pickle.load(handle)

# Load data (deserialize)
with open('test_embeddings_simple_cnn_v2.pkl', 'rb') as handle:
    test_embeddings_v2 = pickle.load(handle)

In [77]:
import pickle
# Load data (deserialize)
with open('train_embeddings_conv_enc_v3.pkl', 'rb') as handle:
    train_embeddings_v3 = pickle.load(handle)

# Load data (deserialize)
with open('val_embeddings_conv_enc_v3.pkl', 'rb') as handle:
    val_embeddings_v3 = pickle.load(handle)

# Load data (deserialize)
with open('test_embeddings_conv_enc_v3.pkl', 'rb') as handle:
    test_embeddings_v3 = pickle.load(handle)

In [78]:
import pickle
# Load data (deserialize)
with open('train_embeddings_conv_enc_v4.pkl', 'rb') as handle:
    train_embeddings_v4 = pickle.load(handle)

# Load data (deserialize)
with open('val_embeddings_conv_enc_v4.pkl', 'rb') as handle:
    val_embeddings_v4 = pickle.load(handle)

# Load data (deserialize)
with open('test_embeddings_conv_enc_v4.pkl', 'rb') as handle:
    test_embeddings_v4 = pickle.load(handle)

In [80]:
train_embeddings_all = []
for train_embedding_v2, train_embedding_v3, train_embedding_v4 in zip(train_embeddings_v2, train_embeddings_v3, train_embeddings_v4):
    concatenated_tensor = torch.cat((train_embedding_v2, train_embedding_v3, train_embedding_v4), dim=0)
    train_embeddings_all.append(concatenated_tensor)

In [81]:
val_embeddings_all = []
for val_embedding_v2, val_embedding_v3, val_embedding_v4 in zip(val_embeddings_v2, val_embeddings_v3, val_embeddings_v4):
    concatenated_tensor = torch.cat((val_embedding_v2, val_embedding_v3, val_embedding_v4), dim=0)
    val_embeddings_all.append(concatenated_tensor)

In [82]:
print(len(train_embeddings_all), len(train_embeddings_all[1]))

13000 4784


In [83]:
print(len(val_embeddings_all), len(val_embeddings_all[1]))

2000 4784


In [25]:
print(len(train_embeddings_ce), len(train_embeddings_ce[1]))

13000 2048


In [26]:
print(len(train_embeddings_v2), len(train_embeddings_v2[1]))

13000 1200


In [27]:
print(len(train_embeddings_v3), len(train_embeddings_v3[1]))

13000 768


In [28]:
print(len(train_embeddings_v4), len(train_embeddings_v4[1]))

13000 768


In [36]:
# Example embeddings and labels
X = np.array(train_embeddings_all)  # image_paths is a list of image file paths
y = np.array(train_image_labels)  # labels for each image

In [37]:
X_val = np.array(val_embeddings_all)  # image_paths is a list of image file paths
y_val = np.array(val_image_labels)

### Train Catboost

In [2]:
# Initialize CatBoost Classifier
catboost_model = CatBoostClassifier(use_best_model=True, loss_function="MultiClass", task_type="GPU", iterations=2000)

# Train the model
catboost_model.fit(X, y, eval_set=(X_val, y_val))

#### Compute Feature Importance

In [42]:
importances = catboost_model.get_feature_importance()

In [46]:
feature_importances = pd.Series(importances, index=[i for i in range(6832)]).sort_values(ascending=False)

In [48]:
feature_importances[:100]

4112    2.149597
4115    1.798838
4822    1.779284
3347    1.548376
4036    1.350348
          ...   
4016    0.295921
4097    0.291356
3330    0.290661
4111    0.282728
4045    0.276977
Length: 100, dtype: float64

### Validation 

#### Catboost

In [39]:
val_pred_labels = catboost_model.predict(val_embeddings_all)

#### Encoder Preds - usually worse than taking the embedding space and using a clasifier

In [360]:
# val_dataset = CustomDataset(val_image_paths, val_image_labels)
# val_data_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

In [356]:
# mean, std = get_mean_std(val_data_loader)
# print(f"Mean: {mean}")
# print(f"Std: {std}")

Mean: tensor([0.6717, 0.5947, 0.6015])
Std: tensor([0.2283, 0.2231, 0.2238])


In [344]:
# transform_val = T.Compose([
#     # T.RandomHorizontalFlip(p=0.5),
#     # T.RandomVerticalFlip(p=0.5),
#     # T.Resize((64, 64)), -> all are already 64 * 64
#     T.ToTensor(),
#     T.Normalize([0.6717, 0.5947, 0.6015], [0.2283, 0.2231, 0.2238])
# ])

In [361]:
# %%time
# val_pred_labels = []
# for images, _ in val_data_loader:
#     images = images.to(device)
#     # labels = labels.to(device)
#     with torch.no_grad():
#         logits = encoder(images)
#         predicted_class = torch.argmax(logits, dim=1)  # Get the index of the max logit
#         val_pred_labels.append(predicted_class.item())  # Return the class index as an integer

#     # optimizer.zero_grad()
#     # loss.backward()
#     # optimizer.step()

# print(f"Loss: {loss.item():.4f}")

Loss: 0.6422
CPU times: user 4.68 s, sys: 623 ms, total: 5.3 s
Wall time: 6.86 s


In [348]:
# def prepare_image(image_path):
#     image = Image.open(image_path)
#     return transform_val(image).unsqueeze(0)  # Add batch dimension

# def predict_class(image_path, model):
#     image_tensor = prepare_image(image_path)
#     image_tensor = image_tensor.to(device)

#     with torch.no_grad():
#         logits = model(image_tensor)
#         predicted_class = torch.argmax(logits, dim=1)  # Get the index of the max logit
#         return predicted_class.item()

In [362]:
# val_pred_labels = [predict_class(path, encoder) for path in val_image_paths]

#### Accuracy

In [40]:
accuracy = accuracy_score(val_image_labels, val_pred_labels)
print(f"Overall Accuracy: {accuracy * 100:.2f}%")

Overall Accuracy: 63.05%


In [161]:
# def top_n_accuracy(true_labels, pred_probs, n=3):
#     best_n = np.argsort(pred_probs, axis=1)[:,-n:]
#     successes = 0
#     for i in range(pred_probs.shape[0]):
#         if true_labels[i] in best_n[i,:]:
#             successes += 1
#     return successes / pred_probs.shape[0]

# # Example usage
# top3_acc = top_n_accuracy(val_image_labels, val_pred_labels, n=3)
# print(f"Top 3 Accuracy: {top3_acc * 100:.2f}%")

Top 3 Accuracy: 1.00%


#### Classification Report

In [143]:
# report = classification_report(val_image_labels, val_pred_labels)
# print(report)

#### CM Analysis

In [41]:
# Generate the confusion matrix
cm = confusion_matrix(val_image_labels, val_pred_labels)

# Summing the diagonal elements gives the total number of correct predictions
correct_predictions = np.trace(cm)
total_predictions = cm.sum()

print(f"Number of Correct Predictions: {correct_predictions}")
print(f"Number of Incorrect Predictions: {total_predictions - correct_predictions}")

Number of Correct Predictions: 1012
Number of Incorrect Predictions: 988


### Train Encoder on Validation

In [30]:
dataset = CustomDataset(val_image_paths, val_image_labels, transform=basic_transform)
data_loader = DataLoader(dataset, batch_size=128, shuffle=True)

In [31]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(encoder.parameters(), lr=0.01)

In [3]:
%%time
num_epochs = 100 
for epoch in range(num_epochs):
    encoder.train()
    for images, labels in data_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        embeddings = encoder(images)
        loss = criterion(embeddings, labels)
    
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

### Train Classification Model on Validation

In [33]:
val_embeddings = [get_embedding(img_path).cpu() for img_path in val_image_paths]

In [4]:
X_val = np.array(val_embeddings)
y_val = np.array(val_image_labels)

catboost_model = CatBoostClassifier(loss_function="MultiClass", task_type="GPU", iterations=3000)

catboost_model.fit(X_val, y_val)

### Predict on Test

In [35]:
test_embeddings = [get_embedding(img_path).cpu() for img_path in test_image_paths]

In [36]:
# test the order
print(test_image_paths[:5])
print()
print(test_df['Image'][:5])

['dataset/test_images/046f61c4-b825-459a-8b2d-07503f5b94a5.png', 'dataset/test_images/67db001f-e287-4950-ac49-6683b493d1a4.png', 'dataset/test_images/9f1d36a1-f046-4c5d-9e8a-0a3758ff605c.png', 'dataset/test_images/5ffef91a-aaf9-4d0d-a219-83a9f5282361.png', 'dataset/test_images/c00af570-0000-4f8f-a3f2-c37a981bfdb1.png']

0    046f61c4-b825-459a-8b2d-07503f5b94a5.png
1    67db001f-e287-4950-ac49-6683b493d1a4.png
2    9f1d36a1-f046-4c5d-9e8a-0a3758ff605c.png
3    5ffef91a-aaf9-4d0d-a219-83a9f5282361.png
4    c00af570-0000-4f8f-a3f2-c37a981bfdb1.png
Name: Image, dtype: object


In [37]:
print(len(test_embeddings_all), len(test_embeddings_all[0]))

5000 6784


In [38]:
X_test = np.array(test_embeddings_all)

In [39]:
y_test = catboost_model.predict(X_test)

In [40]:
test_df['Class'] = y_test

In [41]:
test_df.head()

,Image,Class
0,046f61c4-b825-459a-8b2d-07503f5b94a5.png,67
1,67db001f-e287-4950-ac49-6683b493d1a4.png,82
2,9f1d36a1-f046-4c5d-9e8a-0a3758ff605c.png,40
3,5ffef91a-aaf9-4d0d-a219-83a9f5282361.png,13
4,c00af570-0000-4f8f-a3f2-c37a981bfdb1.png,69


### Create Submission

In [42]:
test_df.to_csv('submission_conv_simple_cnn_v2_conv_enc_v3_v4_and_deep_enc_v1_nov_enc_cb_no_train_valid.csv', index=False)